In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


"c:\program" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [2]:
import pandas as pd
import torch
import transformers
import numpy as np
from tqdm import notebook

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [3]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = transformers.BertModel.from_pretrained('bert-base-uncased')

#config = transformers.BertConfig()

#model = transformers.BertModel.from_pretrained(
#    'bert-base-uncased', config=config)

In [6]:
%%time

df = pd.read_csv('toxic_comments.csv')

#берём только часть данных, а также сбалансируем классы для модели
df_ones = df[df['toxic'] == 1].sample(1000, random_state=123)
df_zeros = df[df['toxic'] == 0].sample(1000, random_state=123)
new_df = shuffle(pd.concat([df_ones] + [df_zeros]))

#df_comments = pd.read_csv('toxic_comments.csv').sample(1000).reset_index(drop=True)
tokenized = new_df['text'].apply(lambda x: tokenizer.encode(x[:512], add_special_tokens=True))

max_len = max(map(len, tokenized)) # вычисляем максимальное кол-во токенов в предложении

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values]) #добавляем 0 для строк, у которых длина меньше максимальной
attention_mask = np.where(padded != 0, 1, 0) # создаем маску, чтобы выделить значимые токены

Wall time: 4.68 s


In [7]:
batch_size = 20 #определяем размер батча для обработки BERT-ом
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:])
        
features = np.concatenate(embeddings) #полученные эмбеддинги будут нашими признаками для обучения модели
target = new_df['toxic']

In [8]:
%%time

# разбиваем на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=123)

#обучим модель логистической регрессии на эмбеддингах
model_lr = LogisticRegression(solver='liblinear', random_state=123)
model_lr.fit(features_train, target_train)

pred_train = model_lr.predict(features_train)
pred_test = model_lr.predict(features_test)

print('F1 обучающей выборки: {:.2f}'.format(f1_score(target_train, pred_train)))
print('F1 тестовой выборки: {:.2f}'.format(f1_score(target_test, pred_test)))

F1 обучающей выборки: 0.97
F1 тестовой выборки: 0.85
Wall time: 438 ms
